# ECON 8601 - IO
## Homework 3 - BLP Exercises
### Xiang Liu
#### Department of Economics, University of Minnesota

I gained a lot of help from Chonghao Hao for these exercises.

In [1]:
using Random, Distributions
using DataStructures
using Base.Threads: @threads
using CSV
using DataFrames
using Optim
using ForwardDiff
using LinearAlgebra
using GLM
using Statistics
using StatsModels

In [2]:
df = CSV.read("cardata.txt", DataFrame; delim='\t');

## Exercise 1 - the Logit Model

Assume utility is given by 
$$u_{ij} = \alpha (y_i - p_j) + x_j\beta + \epsilon_{ij}, \forall j = 0, 1, \cdots, J_t$$
with $\epsilon_{ij}$ type 1 extreme value, $y_i$ income, $x_j, p_j$ observed characteristics, and parameters $\beta, \alpha$.

Derive the appropriate likelihood function for the aggregated data by starting with the micro-level specification and regrouping. 


The utility $\delta_j$ for a product $j$ based on its characteristics $X$ and price $P$ is given by:

$$\delta_j(X, P, \beta, \alpha) = \beta \cdot X - \alpha P$$

The market share $s_j$ for a product $j$, derived from the utility, is calculated as:

$$s_j = \frac{e^{\delta_j}}{1 + \sum_{k} e^{\delta_k}}$$

Given observed market shares $\hat{s}_j$, the objective function for likelihood maximization is:

$$\mathcal{L}(\beta, \alpha) = \sum_{j} \log(s_j) \cdot \hat{s}_j$$

where $\mathcal{L}(\beta, \alpha)$ is to be maximized with respect to $\beta$ and $\alpha$.

Now, in order to do the following exercises, we convert these calculations into functions: 

## Exercise 2 - MLE to the Logit Model

Using the automobile data, estimate the logit demand specification using maximum likelihood and assuming prices are exogenous. What is the implied own-price elaticity of the 1990 Honda Accord (HDACCO)? What is the implied cross-elasticity of the Honda Accord with respect to the 1990 Ford Escort (FDESCO)? Pick two addtional cars and report the same numbers. 

In [3]:
function δ_j(X, P, β, α)
    # X: Vector of characteristics
    # P: Price
    # β: Coefficients for characteristics
    # α: Coefficient for price
    return dot(β, X) - α * P
end

function s_vec(df)
    # no income y
    # δ_vec: Vector of δ_j, j is product    
    # return vector [s_1,...,s_J]' market share for products
    markets = unique(df[!,:year])
    results = OrderedDict()
    @threads for j in markets
        # Perform calculations
        δ = df[df.year .== j, :δ_vec]
        results[j] = exp.(δ) / (1 + sum(exp.(δ)))
    end
    return vcat(values(results)...)
end


function obj_func(s_j, s_hat_j)
    # s_j is estimated market share from s_vec
    # s_hat_j is observed market share for product j
    return log(s_j)*s_hat_j
end

# Function to compute utility for each vehicle
function compute_δ_vec(data, β, α,characteristics;constant=false)
    δ_vec = []
    for i in 1:nrow(data)
        # Extract characteristics and price for each vehicle
        X = [data[i, c] for c in characteristics]
        if constant
            X = [1.0; X]
        end
        P = data[i, :price]

        # Calculate utility and append to δ_vec
        push!(δ_vec, δ_j(X, P, β, α))
    end
    return δ_vec
end

# Function to maximize
function total_log_likelihood(β, α, data, characteristics)
    total_ll = 0.0
    δ_vec = compute_δ_vec(data, β, α,characteristics)
    data[!,:δ_vec] = δ_vec
    s_vec_estimated = s_vec(data)
    # sum over j
    for i in 1:nrow(data)
        total_ll += obj_func(s_vec_estimated[i], data[i, :market_share])
    end
    return total_ll
end



function elasticity(df, car1,car2, year, α_hat)
    # Calculate s_j for the given vehicle and year
    # need to calcualte estimated market share s first
    s_j = df[(df.vehicle_name .== car1) .& (df.year .== year), :s][1] # estimated market share or observed
    s_k = df[(df.vehicle_name .== car2) .& (df.year .== year), :s][1]
    p_j = df[(df.vehicle_name .== car1) .& (df.year .== year), :price][1]
    p_k = df[(df.vehicle_name .== car2) .& (df.year .== year), :price][1]
    # Calculate own and cross price elasticities
    elas_own = p_j/s_j *(-α_hat) * s_j * (1 - s_j)
    elas_cros = p_k/s_j * α_hat * s_k * s_j
    println(car1," Own elasticity: ", round(elas_own, digits=9))
    println(car1," ",car2," Cross elasticity: ", round(elas_cros, digits=9))
    return elas_own, elas_cros
end


elasticity (generic function with 1 method)

In [4]:
# Initial parameter guesses
initial_β = [0.1,0.5, 0.3, 0.2]
initial_α = 0.4
characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]

# Optimization
result = optimize(βα -> -total_log_likelihood(βα[1:end-1], βα[end], df,characteristics), [initial_β..., initial_α])

# Extract optimized parameters
optimized_parameters = Optim.minimizer(result)
println("Optimized β: ", join(round.(optimized_parameters[1:end-1], digits=4), ", "))
println("Optimized α: ", round(optimized_parameters[end], digits=4))


Optimized β: -0.044, 1.884, 0.18, 0.1834
Optimized α: 0.1357


In [5]:
H = ForwardDiff.hessian(βα -> -total_log_likelihood(βα[1:end-1], βα[end], df,characteristics), optimized_parameters)
se = sqrt.(diag(inv(H)))
println("SE β: ", join(round.(se[1:end-1], digits=4), ", "))
println("SE α: ", round(se[end], digits=4))

SE β: 11.1, 3.9348, 3.1169, 1.903
SE α: 0.327


In [6]:
β_hat = optimized_parameters[1:end-1]
α_hat = optimized_parameters[end]
δ_vec = compute_δ_vec(df, β_hat, α_hat,characteristics)
df[!,:δ_vec] = δ_vec
s_vec_estimated = s_vec(df)
df[!, :s] = s_vec_estimated;

In [7]:
elas_own, elas_cros = elasticity(df, "HDACCO","FDESCO", 90, α_hat)
elas_own, elas_cros = elasticity(df, "CRDYNA","CRLANC", 89, α_hat)
elas_own, elas_cros = elasticity(df, "CHSUMM","CHTC", 89, α_hat);

HDACCO Own elasticity: -1.249041902
HDACCO FDESCO Cross elasticity: 0.011376034
CRDYNA Own elasticity: -1.333302909
CRDYNA CRLANC Cross elasticity: 0.011559582
CHSUMM Own elasticity: -1.0125512
CHSUMM CHTC Cross elasticity: 0.003007771


## Exercise 3 - Linear Regression of the Logit Model

Estimate the logit demand specification using the linearized version of this model from BLP (i.e. regress $\log (s_j - s_0)$ on characteristics). What is the implied own-prince elaticity of the 1990 Honda Accord (HDACCO)? What is the implied cross-elasticity of the Honda Accord with respect to the 1990 Ford Escort (FDESCO)? Pick two addtional cars and report the same numbers. 

In [8]:

function ols_reg(Y,X)
    β = (X'X) \ (X'Y)
    err = Y - X * β
    # Estimate of error variance
    σ²_hat = (err' * err)/(size(X,1)-size(X,2))

    # Variance-Covariance matrix of β_hat
    var_beta = σ²_hat * inv(X'X)

    se_beta = sqrt.(diag(var_beta))

    err = Y - X * β
    Xϵ = X .* (err * ones(1, size(X, 2)))
    var_beta = inv(X'*X)* Xϵ' * Xϵ * inv(X'*X)
    se_beta = sqrt.(diag(var_beta))
    return β, se_beta
end

ols_reg (generic function with 1 method)

In [9]:
# Calcualte s_0 for each year
grouped_df = groupby(df, :year)
market_share_sum = combine(grouped_df, :market_share => (x -> 1 - sum(x)) => :s_0)
df = leftjoin(df, market_share_sum, on=:year);

In [10]:
X = hcat(ones(size(df, 1)), Matrix(df[!, [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar, :price]]))

# Create the vector Y for the dependent variable
Y = log.(df[!, :market_share]) - log.(df[!, :s_0])

# Perform the OLS regression using the Normal Equation: β = (X'X)^(-1)X'Y
β, se_beta = ols_reg(Y,X)
β_hat = β[1:end-1]
α_hat = -β[end]

println("β_hat: ", join(round.(β_hat, digits=4), ", "))
println("α_hat: ", round(α_hat, digits=4))
println("SE β: ", join(round.(se_beta[1:end-1], digits=4), ", "))
println("SE α: ", round(se_beta[end], digits=4))

β_hat: -10.0716, -0.1242, 2.3421, -0.0343, 0.265
α_hat: 0.0886
SE β: 0.2572, 0.2786, 0.1244, 0.0709, 0.0424
SE α: 0.0043


In [11]:
δ_vec = compute_δ_vec(df, β_hat, α_hat, characteristics,constant= true)
df[!,:δ_vec] = δ_vec
s_vec_estimated = s_vec(df)
df[!, :s] = s_vec_estimated

elas_own, elas_cros = elasticity(df, "HDACCO","FDESCO", 90, α_hat)
elas_own, elas_cros = elasticity(df, "CRDYNA","CRLANC", 89, α_hat)
elas_own, elas_cros = elasticity(df, "CHSUMM","CHTC", 89, α_hat);

HDACCO Own elasticity: -0.823079102
HDACCO FDESCO Cross elasticity: 0.000451967
CRDYNA Own elasticity: -0.878261183
CRDYNA CRLANC Cross elasticity: 0.000526359
CHSUMM Own elasticity: -0.667692045
CHSUMM CHTC Cross elasticity: 0.000228044


## Exercise 4.1 - BLP Instruments in Logit Model

Write

$$u_{ij} = \alpha \ln(y_i – p_j) + x_j \beta + \xi_j + \epsilon_{ij}, \forall j = 0, 1, \cdots, J.$$

Use the instruments used in BLP. You will nned the firmids and the year variables to calculate  these instruments (they are product-firm-year specific). Estimate the logit model using 2SLS and instrumenting for price. 

What is the implied own-prince elaticity of the 1990 Honda Accord (HDACCO)? What is the implied cross-elasticity of the Honda Accord with respect to the 1990 Ford Escort (FDESCO)? Pick two addtional cars and report the same numbers. 



In [12]:
function construct_instrument(data::DataFrame, characteristic::Symbol)
    own_firm_sum = Symbol("own_firm_sum_", characteristic)
    rival_firm_sum = Symbol("rival_firm_sum_", characteristic)
    data[!, own_firm_sum] = zeros(nrow(data))
    data[!, rival_firm_sum] = zeros(nrow(data))
    for i in 1:nrow(data)
        current_firm = data[i, :firmid]
        current_time = data[i, :year]

        # Filter data for the current firm and time period
        own_firm_data = data[(data[!, :firmid] .== current_firm) .& (data[!, :year] .== current_time), :]
        rival_firm_data = data[(data[!, :firmid] .!= current_firm) .& (data[!, :year] .== current_time), :]

        # Sum of characteristic for own-firm products (excluding current product)
        data[i, own_firm_sum] = sum(own_firm_data[!, characteristic]) - data[i, characteristic]

        # Sum of characteristic for rival-firm products
        data[i, rival_firm_sum] = sum(rival_firm_data[!, characteristic])
    end
end

construct_instrument (generic function with 1 method)

In [13]:
function IV2sls(Y, X_endogenous,X_exogenous, Z)
    # First Stage: Regress X on Z
    B_hat = (Z'Z) \ (Z'X_endogenous)  # Estimated coefficients for the instrumental variables
    X_hat = Z * B_hat       # Predicted values of X

    X_combined = hcat(X_exogenous,X_hat)
    # Second Stage: Regress Y on X_hat
    beta_2sls = (X_combined'X_combined) \ (X_combined'Y)
    
    err = Y - X_combined * beta_2sls
    Xϵ = X_combined .* (err * ones(1, size(X_combined, 2)))
    var_beta = inv(X_combined'*X_combined)* Xϵ' * Xϵ * inv(X_combined'*X_combined)
    se_beta = sqrt.(diag(var_beta))

    return beta_2sls, se_beta
end

IV2sls (generic function with 1 method)

In [14]:
# Construct BLP IV
characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]
for char in characteristics
    construct_instrument(df, char)
end
# Define the instrumental variables
instruments = vcat(characteristics,
                [Symbol("own_firm_sum_", char) for char in characteristics],
                [Symbol("rival_firm_sum_", char) for char in characteristics] ) ;

In [15]:
# Convert DataFrame columns to matrices
Y = log.(df[!, :market_share]) - log.(df[!, :s_0])
constant = ones(size(df, 1))
X_endogenous = Matrix(df[:, [:price]])

X_exogenous = Matrix(df[:, characteristics])
X_exogenous = hcat(constant, X_exogenous)

Z = Matrix(df[:, instruments])
Z = hcat(constant, Z);

In [16]:
result,se = IV2sls(Y, X_endogenous,X_exogenous, Z)

β_hat = result[1:end-1]
α_hat = result[end]

println("β_hat: ", join(round.(β_hat, digits=3), ", "))
println("α_hat: ", round(α_hat, digits=3))
println("SE β: ", join(round.(se_beta[1:end-1], digits=3), ", "))
println("SE α: ", round(se_beta[end], digits=3))

β_hat: -9.902, 1.345, 2.287, 0.532, 0.163
α_hat: -0.14
SE β: 0.257, 0.279, 0.124, 0.071, 0.042
SE α: 0.004


In [17]:
δ_vec = compute_δ_vec(df, β_hat, α_hat, characteristics,constant= true)
df[!,:δ_vec] = δ_vec
s_vec_estimated = s_vec(df)
df[!, :s] = s_vec_estimated

elas_own, elas_cros = elasticity(df, "HDACCO","FDESCO", 90, α_hat)
elas_own, elas_cros = elasticity(df, "CRDYNA","CRLANC", 89, α_hat)
elas_own, elas_cros = elasticity(df, "CHSUMM","CHTC", 89, α_hat);

HDACCO Own elasticity: 1.29930615
HDACCO FDESCO Cross elasticity: -9.3166e-5
CRDYNA Own elasticity: 1.386705671
CRDYNA CRLANC Cross elasticity: -4.1518e-5
CHSUMM Own elasticity: 1.054232826
CHSUMM CHTC Cross elasticity: -0.002871674


## Exercise 4.2 - Gandhi-Houde (DH) Instrumnets in Logit Model

In [18]:
# Function to calculate GH IV, I only construct quadratic instruments without interaction terms
function GH_IV(df, characteristics)
    grouped_data = groupby(df, :year)
    collected_groups = []
    for (market, group) in pairs(grouped_data)
        firm_intra_distances = Dict()
        firm_inter_distances = Dict()
    
        for char in characteristics
            n = size(group, 1)
            intra_distance_matrix = zeros(n, n)
            inter_distance_matrix = zeros(n, n)
        
            for i in 1:n
                for j in 1:n
                    if group.firmid[i] == group.firmid[j]  # Intrafirm distance
                        intra_distance_matrix[i, j] = group[i, char] - group[j, char]
                    elseif group.firmid[i] != group.firmid[j]  # Interfirm distance
                        inter_distance_matrix[i, j] = group[i, char] - group[j, char]
                    end
                end
            end
        
            firm_intra_distances[char] = intra_distance_matrix.^ 2
            firm_inter_distances[char] = inter_distance_matrix.^ 2
        end
        group = convert_IV_df(firm_intra_distances,group,"intra_IV_")
        group = convert_IV_df(firm_inter_distances,group,"inter_IV_")
        push!(collected_groups, group)
    end
    
    final_df = vcat(collected_groups...)

    return final_df 
end

function convert_IV_df(firm_intra_distances, group, prefix)
    row_sums = [sum(firm_intra_distances[char], dims=2) for char in keys(firm_intra_distances)]
    transposed_sums = hcat(row_sums...)
    # Adding new columns to group
    for (i, key) in enumerate(keys(firm_intra_distances))
        column_name = Symbol(prefix * string(key))
        group[!, column_name] = transposed_sums[:, i]
    end
    return group
end

convert_IV_df (generic function with 1 method)

In [19]:
# construct GH_IV
df = GH_IV(df, characteristics)
GH_instruments = vcat(characteristics,
                [Symbol("intra_IV_", char) for char in characteristics],
                [Symbol("inter_IV_", char) for char in characteristics] ) ;

In [20]:

Z = Matrix(df[:, GH_instruments])
Z = hcat(constant, Z)

result,se = IV2sls(Y, X_endogenous,X_exogenous, Z)

β_hat = result[1:end-1]
α_hat = -result[end]

println("β_hat: ", join(round.(β_hat, digits=3), ", "))
println("α_hat: ", round(α_hat, digits=3))
println("SE β: ", join(round.(se_beta[1:end-1], digits=3), ", "))
println("SE α: ", round(se_beta[end], digits=3))

δ_vec = compute_δ_vec(df, β_hat, α_hat, characteristics,constant= true)
df[!,:δ_vec] = δ_vec
s_vec_estimated = s_vec(df)
df[!, :s] = s_vec_estimated

elas_own, elas_cros = elasticity(df, "HDACCO","FDESCO", 90, α_hat)
elas_own, elas_cros = elasticity(df, "CRDYNA","CRLANC", 89, α_hat)
elas_own, elas_cros = elasticity(df, "CHSUMM","CHTC", 89, α_hat);

β_hat: -9.908, 1.292, 2.289, 0.512, 0.167
α_hat: 0.138
SE β: 0.257, 0.279, 0.124, 0.071, 0.042
SE α: 0.004
HDACCO Own elasticity: -1.281617801
HDACCO FDESCO Cross elasticity: 0.000699002
CRDYNA Own elasticity: -1.367689576
CRDYNA CRLANC Cross elasticity: 0.000703968
CHSUMM Own elasticity: -1.039727024
CHSUMM CHTC Cross elasticity: 0.000353852


## Exercise 5.1 - BLP Instruments in Logit Model with Random Coefficients

Write

$$u_{ij} = \alpha (y_i – p_j) + x_j \beta_i + \xi_j + \epsilon_{ij}, \forall j = 0, 1, \cdots, J.$$

Now add random coefficients for each characteristic and estimate the means and variances of these nromally distributed random coefficients. Estimate the demand side of the model only (unlesss you are ambitious and want smaller standard errors - then add the supply side too).

What is the implied own-prince elaticity of the 1990 Honda Accord (HDACCO)? What is the implied cross-elasticity of the Honda Accord with respect to the 1990 Ford Escort (FDESCO)? Pick two addtional cars and report the same numbers. 

This part is unfinished, see Julia version for random coefficients. 

In [21]:
function gen_random(characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar ])
    # Number of people
    N = 100

    # Create an empty DataFrame
    people_df = DataFrame() 
    # Random number generator from normal distribution
    rng = MersenneTwister()
    normal_dist = Normal(0, 1)

    # Generate and add data for each characteristic
    for char in characteristics
        people_df[!, char] = rand(rng, normal_dist, N)
    end
    return people_df
end

# char_matrix = Matrix(df[:, characteristics])

#Function for estimated share for one market
function eshare(char_matrix::Matrix, δ::Vector, rand_sample::Matrix, σ::Vector) 
    # δ is the vector of δ_i in market t
    # σ scaling matrix for v
    # rand_sample is the random coefficient of each coefficient for each person [v_i1,..,v_ik] k characteristics
    R = size(rand_sample,1) # #. of simulated individuals
    k = size(char_matrix,1) # #. of products
    share = zeros(k)
    
    # s_jt = 1/R * sum_i{ exp(δ_jt + μ_ijt)/sum_j'{exp(δ_j't + μ_ij't)} }
    for i in 1:R
        # exp(δ_jt + μ_ijt) where μ_ijt = σ_1*v_i1*char_1 + ... + σ_k*v_ik*char_k
        numerator = exp.(δ + char_matrix * (rand_sample[i, :] .* σ)) 
        denominator = 1 + sum(numerator)
        prob = numerator./denominator
        share += 1/R * prob
    end
    return share 
end

#Function for estimating delta for one market
function cdelta(char_matrix::Matrix, rand_sample::Matrix,share::Vector,σ::Vector; tol::Float64 = 1e-12)
    k = size(char_matrix, 1)
    δ_0 = ones(k)
    δ_1 = ones(k)
    error = 1

    while error > tol
        δ_0 = δ_1
        estimate_share = eshare(char_matrix, δ_0, rand_sample,σ)
        δ_1 = δ_0 + log.(share) - log.(estimate_share)
        error = (δ_1 - δ_0)' * (δ_1 - δ_0)
    end

    return δ_1
end

#   Estimate delta for all the markets
#   characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]
function delta_full(df,rand_sample, characteristics, σ ; tol::Float64 = 1e-12)
    markets = unique(df[:,:year])
    results = OrderedDict()
    err = OrderedDict()
    @threads for j in markets
        # Perform calculations
        chars = Matrix(df[df.year .== j, characteristics])
        share = df[df.year .== j, :market_share]
        δ_new = cdelta(chars, rand_sample, share, σ; tol)
        results[j] = δ_new
        if any(isnan, δ_new)
            err[j] = j
        end
    end
    return vcat(values(results)...), err
end

# Objective function, get GMM error and β
# rand_sample= Matrix(gen_random())
# characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar ]

function obj(df,rand_sample,characteristics,IV_list,σ;delta_full_func=delta_full)
    δ = delta_full_func(df,rand_sample,characteristics,σ)[1]
    z1 = Matrix(df[: , IV_list])
    chars = characteristics
    if !(:price in characteristics) 
        chars = [characteristics..., :price]
    end    
    x = hcat(ones(size(df, 1)), Matrix(df[!, chars]))
    z = [ones(size(df, 1)) z1] # IV matrix with constant term
    w = inv(z' * z)
    beta = (x'* z * w * z' * x)^-1  * (x'* z * w * z' * δ)
    u = δ - x * beta
    res = u' * z * w * z' * u
    return (res, beta, u)
end

function random_elasticity(df, car1, car2, yr, rand_sample, σ, β, characteristics)
    group = df[df.year .== yr,:]
    car_j_ind = findfirst(group.vehicle_name.== car1) 
    car_k_ind = findfirst(group.vehicle_name.== car2)
    p_j = group[car_j_ind, :price]
    p_k = group[car_k_ind, :price]
    s_j = group[car_j_ind, :market_share]
    α = -β[end]

    ns = size(rand_sample,1)
    own = 0
    cross = 0
    s_j_hat= 0
    chars = characteristics
    if !(:price in characteristics) 
        chars = [characteristics..., :price]
    end
    char_matrix_j = Matrix(df[df.year .== yr, chars])
    for i in 1:ns
        if :price in characteristics # can also input two characteristics list for δ and μ
            μ_ijt = char_matrix_j * (rand_sample[i, :] .* σ)
        else
            μ_ijt = char_matrix_j[:, 1:end-1] * (rand_sample[i, :] .* σ)
        end
     
        δ_ijt = char_matrix_j * β[2:end] .+ β[1]
        
        num = exp.( δ_ijt + μ_ijt )
        den = 1 + sum(num)
        s_i = num./den

        s_ij = s_i[car_j_ind]
        s_ik = s_i[car_k_ind]

        s_j_hat += 1/ns *s_i[car_j_ind]
        
        own += 1/ns * (-α) * s_ij * (1 - s_ij)
        cross += 1/ns * α * s_ij * s_ik
    end
    elas_own = own * p_j / s_j_hat
    elas_cros = cross * p_k / s_j_hat
    println(car1," Own elasticity: ", round(elas_own, digits=9))
    println(car1," ",car2," Cross elasticity: ", round(elas_cros, digits=9))
    return elas_own, elas_cros
end


random_elasticity (generic function with 1 method)

In [22]:
# In HW, there is no random coefficient for price
characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]
rand_sample= Matrix(gen_random(characteristics))
for char in characteristics # avoid inlcuding price itself in IV list
    construct_instrument(df, char)
end
IV_list = vcat(characteristics,
                [Symbol("own_firm_sum_", char) for char in characteristics],
                [Symbol("rival_firm_sum_", char) for char in characteristics] ) ;

In [23]:
σ_0 = [1.0,1.0,1.0,1.0] # random coefficient initial guess
result1 = optimize(sig -> obj(df,rand_sample,characteristics,IV_list,sig)[1], σ_0);

In [59]:
# β 
optimized_parameters = Optim.minimizer(result1)
_,β,gmm_s = obj(df,rand_sample,characteristics,IV_list,optimized_parameters)
println("β: ", join(round.(β, digits=3), ", "))
σ = optimized_parameters = Optim.minimizer(result1)
println("σ: ", join(round.(σ, digits=3), ", "))
θ1_se, θ2_se = varcov(df,IV_list, σ, gmm_s)
println("SE β: ", join(round.(θ1_se, digits=4), ", "))
println("SE σ: ", join(round.(θ2_se, digits=4), ", "))

TaskFailedException: TaskFailedException

    nested task error: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 5 and 4
    Stacktrace:
      [1] _bcs1
        @ ./broadcast.jl:516 [inlined]
      [2] _bcs
        @ ./broadcast.jl:510 [inlined]
      [3] broadcast_shape
        @ ./broadcast.jl:504 [inlined]
      [4] combine_axes
        @ ./broadcast.jl:499 [inlined]
      [5] instantiate
        @ ./broadcast.jl:281 [inlined]
      [6] materialize
        @ ./broadcast.jl:860 [inlined]
      [7] eshare(char_matrix::Matrix{Float64}, δ::Vector{Float64}, rand_sample::Matrix{Float64}, σ::Vector{Float64})
        @ Main ~/Desktop/IO-Petrin/BLP_Exercises.ipynb:32
      [8] cdelta(char_matrix::Matrix{Float64}, rand_sample::Matrix{Float64}, share::Vector{Float64}, σ::Vector{Float64}; tol::Float64)
        @ Main ~/Desktop/IO-Petrin/BLP_Exercises.ipynb:49
      [9] macro expansion
        @ ~/Desktop/IO-Petrin/BLP_Exercises.ipynb:67 [inlined]


In [25]:
random_elasticity(df, "HDACCO","FDESCO", 90, rand_sample, σ, β, characteristics)
random_elasticity(df, "CRDYNA","CRLANC", 89, rand_sample, σ, β, characteristics)
random_elasticity(df, "CHSUMM","CHTC", 89, rand_sample, σ, β, characteristics);

HDACCO Own elasticity: -1.489458595
HDACCO FDESCO Cross elasticity: 0.005413356
CRDYNA Own elasticity: -1.593253645
CRDYNA CRLANC Cross elasticity: 0.00336622
CHSUMM Own elasticity: -1.210611003
CHSUMM CHTC Cross elasticity: 0.002967242


## Exercise 5.2 - Gandhi-Houde (DH) Instrumnets in Logit Model with Random Coefficients

In [26]:
σ_0 = [1.0,1.0,1.0,1.0] # random coefficient initial guess
result1 = optimize(sig -> obj(df,rand_sample,characteristics,GH_instruments,sig)[1], σ_0);

In [27]:
# β 
optimized_parameters = Optim.minimizer(result1)
_,β,gmm_s = obj(df,rand_sample,characteristics,IV_list,optimized_parameters)
println("β: ", join(round.(β, digits=3), ", "))
σ = optimized_parameters = Optim.minimizer(result1)
println("σ: ", join(round.(σ, digits=3), ", "))
θ1_se, θ2_se = varcov(df,IV_list, σ, gmm_s)
println("SE β: ", join(round.(θ1_se, digits=4), ", "))
println("SE σ: ", join(round.(θ2_se, digits=4), ", "))

β: -7.392, -19.171, 2.951, 0.78, -0.237, -0.184
σ: 12.044, -0.558, -0.687, 0.616


UndefVarError: UndefVarError: varcov not defined

In [28]:
random_elasticity(df, "HDACCO","FDESCO", 90, rand_sample, σ, β, characteristics)
random_elasticity(df, "CRDYNA","CRLANC", 89, rand_sample, σ, β, characteristics)
random_elasticity(df, "CHSUMM","CHTC", 89, rand_sample, σ, β, characteristics);

HDACCO Own elasticity: -1.697696219
HDACCO FDESCO Cross elasticity: 0.017753898
CRDYNA Own elasticity: -1.816037114
CRDYNA CRLANC Cross elasticity: 0.008523808
CHSUMM Own elasticity: -1.37618383
CHSUMM CHTC Cross elasticity: 0.002309367


## Exercise 6.1 - BLP Instruments with Another Random Coefficient - Income effects

Write

$$u_{ij} = \alpha \ln(y_i – p_j) + x_j \beta_i + \xi_j + \epsilon_{ij}, \forall j = 0, 1, \cdots, J.$$

Draw $y_i$ from a log-normal with a mean that varies by year for 1971-1990 given by (2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745, 2.09805, 2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426, 2.18071, 2.18856, 2.21250, 2.18377), and a fixed variance given by $\sigma_y = 1.72$. 

What is the implied own-prince elaticity of the 1990 Honda Accord (HDACCO)? What is the implied cross-elasticity of the Honda Accord with respect to the 1990 Ford Escort (FDESCO)? Pick two addtional cars and report the same numbers. 

This part is unfinished, see Julia version for random coefficients. 

In [29]:
# Mean values for each year from 1971 to 1990
function income_sample()
    mean_values = [
        2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745, 2.09805, 
        2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426, 
        2.18071, 2.18856, 2.21250, 2.18377
    ]

    # Fixed standard deviation (square root of variance)
    sigma_y = sqrt(1.72)
    years = 71:90  # Years from 1971 to 1990

    # Dictionary to store the samples for each year
    samples_by_year = Dict()

    for (year, mean) in zip(years, mean_values)
        # Drawing 100 samples from a log-normal distribution for each year
        distribution = LogNormal(mean, sigma_y)
        samples = rand(distribution, 100)
        samples_by_year[year] = samples
    end
    return samples_by_year
end

# rewrite delta_full to include income effect
# people_income =income_sample()
function delta_full_income(df, rand_sample, characteristics, σ, people_income ; tol::Float64 = 1e-20)
    markets = unique(df[:,:year])
    results = OrderedDict()
    err = OrderedDict()
    @threads for j in markets
        rand_sample[:, end] = 1/people_income[j]
        chars = Matrix(df[df.year .== j, characteristics])
        share = df[df.year .== j, :market_share]
        δ_new = cdelta(chars, rand_sample, share, σ)
        results[j] = δ_new

        if any(isnan, δ_new)
            err[j] = j
        end
    end
    return vcat(values(results)...), err
end

function income_obj(df,rand_sample,characteristics,IV_list,σ,people_income)
    δ = delta_full_income(df, rand_sample, characteristics, σ, people_income)[1]
    z1 = Matrix(df[: , IV_list])
    chars = characteristics
    if !(:price in characteristics) 
        chars = [characteristics..., :price]
    end
    x = hcat(ones(size(df, 1)), Matrix(df[!, chars]))
    z = [ones(size(df, 1)) z1] # IV matrix with constant term
    w = inv(z' * z)
    beta = (x'* z * w * z' * x)^-1  * (x'* z * w * z' * δ)
    u = δ - x * beta
    res = u' * z * w * z' * u
    return (res, beta, u)
end

income_obj (generic function with 1 method)

In [30]:
# Estimate SE of parameters (mini van paper p17)
# Petrin's code use small change to estimate Jacobian matrix d(g)/dθ_2 where g is GMM objective function g'g = u' * z * w * z' * u
# I use same method to calculate Jacobian matrix dδ/dθ_2, they should be same theoretically
function jacobian_matrix(df,σ)
    # σ is the random coefficient estimated by dynamic programming
    n = nrow(df)
    kparm = size(σ,1)
    dδ = zeros(n, kparm)
    imat = I(kparm)
    δ_0 = delta_full(df,rand_sample, characteristics, σ)[1]
    epsilon = 1e-8

    for i in 1:kparm
        # finding derivative for parameter
        dp = epsilon * abs(σ[i])
        dp = max(epsilon, dp)
        parm = σ + dp * imat[:, i]
        δ = delta_full(df,rand_sample, characteristics, parm)[1]
        dδ[:, i] = (δ - δ_0) / dp
    end
    return dδ
end

function varcov(df,IV_list, σ, gmm_s)
    # σ is the random coefficient estimated by dynamic programming
    # gmm_s is the GMM residual u = δ - x * beta
    z1 = Matrix(df[: , IV_list])
    chars = characteristics
    if !(:price in characteristics) # if price doesn't have random coefficient, still need to add it into X
        chars = [characteristics..., :price]
    end    
    x = hcat(ones(size(df, 1)), Matrix(df[!, chars])) # characteristics matrix
    z = [ones(size(df, 1)) z1] # IV matrix
    w = inv(z' * z)
    temp = jacobian_matrix(df,σ)
    a = hcat(x, temp)' * z
    IVres = z .* (gmm_s * ones(1, size(z, 2)))
    b = IVres' * IVres
    inv_aAa = inv(a * w * a')
    f = inv_aAa * (a * w * b * w * a')* inv_aAa 
    # same as f = gmm_s'*gmm_s/size(x,1)*inv(a*w*a') if W = inv(z'z) and homoskedastic
    diagonal = diag(f)
    θ1_se = diagonal[1:size(x,2)]
    θ2_se = diagonal[size(x,2)+1:end]
    return θ1_se, θ2_se
end





varcov (generic function with 1 method)

In [31]:
people_income =income_sample()

Dict{Any, Any} with 20 entries:
  78 => [42.5853, 12.6654, 23.9363, 8.12771, 4.49778, 7.49059, 66.1431, 238.546…
  79 => [7.34236, 2.49757, 19.3833, 7.41136, 4.36736, 2.63066, 4.96517, 11.968,…
  81 => [0.407758, 28.8697, 17.5023, 22.7314, 0.724975, 62.1736, 15.2102, 86.37…
  72 => [8.22064, 16.6072, 0.783041, 7.24049, 4.31433, 0.567107, 43.9141, 10.31…
  75 => [2.75934, 1.99328, 7.56706, 62.4355, 14.2879, 5.4496, 1.30103, 12.2221,…
  83 => [8.98621, 30.0901, 105.018, 4.54441, 7.0594, 4.11634, 28.5386, 9.96706,…
  73 => [81.8484, 10.4901, 24.4546, 5.78342, 21.4674, 6.52388, 18.302, 1.34654,…
  82 => [0.576339, 32.0561, 1.03855, 2.15775, 4.90863, 17.3833, 44.6335, 44.587…
  85 => [2.46732, 17.1592, 18.4833, 15.1701, 3.98941, 128.417, 2.75351, 6.46768…
  74 => [19.8916, 1.77449, 10.497, 4.14141, 29.8833, 29.2845, 34.8155, 29.8697,…
  89 => [23.0835, 6.34252, 3.44363, 10.922, 25.4347, 4.61556, 9.12199, 51.6719,…
  80 => [5.8822, 16.9893, 1.49765, 8.99487, 1.45074, 9.11495, 42.6264, 47.158

In [32]:
σ_0 = [1.0,1.0,1.0,1.0,5.0]
characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]
rand_sample= Matrix(gen_random(characteristics))
push!(characteristics, :price)
rand_sample = [rand_sample zeros(size(rand_sample, 1))]
result = optimize(sig -> income_obj(df,rand_sample,characteristics,IV_list,sig,people_income)[1], σ_0);

In [33]:
optimized_parameters = Optim.minimizer(result)
_,β,gmm_s = income_obj(df,rand_sample,characteristics,IV_list,optimized_parameters,people_income)
println("β: ", join(round.(β, digits=3), ", "))
σ = optimized_parameters = Optim.minimizer(result)
println("σ: ", join(round.(σ, digits=3), ", "))
θ1_se, θ2_se = varcov(df,IV_list, σ, gmm_s)
println("SE β: ", join(round.(θ1_se, digits=4), ", "))
println("SE σ: ", join(round.(θ2_se, digits=4), ", "))

β: -7.088, 0.154, -1.622, -0.134, 0.079, -0.557
σ: 5.756, 4.043, 4.599, -0.418, 405.291
SE β: 0.5002, 77.5705, 2.641, 6.7782, 0.0319, 0.0194
SE σ: 36.4416, 1.8973, 8.9036, 0.1184, 13146.794


In [34]:
rand_sample[:, end] = 1/people_income[90]
random_elasticity(df, "HDACCO","FDESCO", 90, rand_sample, σ, β, characteristics)
rand_sample[:, end] = 1/people_income[89]
random_elasticity(df, "CRDYNA","CRLANC", 89, rand_sample, σ, β, characteristics)
random_elasticity(df, "CHSUMM","CHTC", 89, rand_sample, σ, β, characteristics);

HDACCO Own elasticity: -5.129617319
HDACCO FDESCO Cross elasticity: 0.081087211
CRDYNA Own elasticity: -5.497245064
CRDYNA CRLANC Cross elasticity: 0.021791285
CHSUMM Own elasticity: -4.167381694
CHSUMM CHTC Cross elasticity: 3.7592e-5


## Exercise 6.2 - GH Instruments with Another Random Coefficient - Income effects

In [35]:
people_income =income_sample()

Dict{Any, Any} with 20 entries:
  78 => [1.0846, 12.675, 58.8975, 19.5848, 9.4326, 1.42051, 19.1979, 42.4415, 6…
  79 => [2.25997, 7.85469, 5.86357, 2.17081, 74.8647, 72.7033, 2.02702, 20.1384…
  81 => [6.85626, 7.17855, 7.91857, 1.14936, 1.01764, 3.82252, 25.6168, 4.86221…
  72 => [3.5571, 9.91109, 6.56954, 8.44957, 3.67692, 2.1771, 248.496, 2.08976, …
  75 => [8.00995, 13.8531, 8.818, 11.1757, 5.24543, 1.95792, 3.25159, 1.33702, …
  83 => [18.4723, 3.44382, 9.85863, 5.6754, 52.154, 1.65323, 9.68304, 10.514, 9…
  73 => [3.77468, 1.16999, 5.84494, 1.91996, 5.28611, 29.6764, 44.6928, 8.95743…
  82 => [1.71069, 10.2709, 1.9231, 15.7714, 8.52535, 6.45478, 231.969, 17.6084,…
  85 => [17.3323, 0.780785, 16.5509, 20.2016, 1.19888, 1.46592, 301.95, 8.76774…
  74 => [1.82492, 25.7572, 6.87549, 2.94067, 3.12633, 69.5877, 6.14096, 35.7675…
  89 => [11.8361, 13.1859, 7.59315, 5.27697, 1.94431, 30.7185, 3.47646, 9.94662…
  80 => [5.0985, 16.678, 12.7955, 30.1104, 16.5548, 9.99851, 4.12778, 1.85066

In [36]:
σ_0 = [1.0,1.0,1.0,1.0,1.0]
characteristics = [:horsepower_weight, :length_width, :ac_standard, :miles_per_dollar]
rand_sample= Matrix(gen_random(characteristics))
push!(characteristics, :price)
rand_sample = [rand_sample zeros(size(rand_sample, 1))]
result = optimize(sig -> income_obj(df,rand_sample,characteristics,GH_instruments,sig,people_income)[1], σ_0);

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] Array
        @ ./boot.jl:459 [inlined]
      [2] Array
        @ ./boot.jl:468 [inlined]
      [3] Array
        @ ./boot.jl:476 [inlined]
      [4] similar
        @ ./abstractarray.jl:841 [inlined]
      [5] similar
        @ ./abstractarray.jl:840 [inlined]
      [6] similar
        @ ./broadcast.jl:212 [inlined]
      [7] similar
        @ ./broadcast.jl:211 [inlined]
      [8] copy
        @ ./broadcast.jl:885 [inlined]
      [9] materialize
        @ ./broadcast.jl:860 [inlined]
     [10] broadcast_preserving_zero_d
        @ ./broadcast.jl:849 [inlined]
     [11] *(A::Float64, B::Vector{Float64})
        @ Base ./arraymath.jl:21
     [12] eshare(char_matrix::Matrix{Float64}, δ::Vector{Float64}, rand_sample::Matrix{Float64}, σ::Vector{Float64})
        @ Main ~/Desktop/IO-Petrin/BLP_Exercises.ipynb:35
     [13] cdelta(char_matrix::Matrix{Float64}, rand_sample::Matrix{Fl

In [37]:
optimized_parameters = Optim.minimizer(result)
_,β,gmm_s = income_obj(df,rand_sample,characteristics,IV_list,optimized_parameters,people_income)
println("β: ", join(round.(β, digits=3), ", "))
σ = optimized_parameters = Optim.minimizer(result)
println("σ: ", join(round.(σ, digits=3), ", "))
θ1_se, θ2_se = varcov(df,IV_list, σ, gmm_s)
println("SE β: ", join(round.(θ1_se, digits=4), ", "))
println("SE σ: ", join(round.(θ2_se, digits=4), ", "))

β: -8.101, -5.368, -1.502, -2.25, 0.567, -0.461
σ: 5.756, 4.043, 4.599, -0.418, 405.291
SE β: 2.6015, 25.2418, 22.8594, 185.0102, 0.3821, 0.0359
SE σ: 8.0121, 11.4908, 157.5038, 1.0607, 8045.4211


In [38]:
rand_sample[:, end] = 1/people_income[90]
random_elasticity(df, "HDACCO","FDESCO", 90, rand_sample, σ, β, characteristics)
rand_sample[:, end] = 1/people_income[89]
random_elasticity(df, "CRDYNA","CRLANC", 89, rand_sample, σ, β, characteristics)
random_elasticity(df, "CHSUMM","CHTC", 89, rand_sample, σ, β, characteristics);

HDACCO Own elasticity: -4.243999414
HDACCO FDESCO Cross elasticity: 0.060064859
CRDYNA Own elasticity: -4.54984679
CRDYNA CRLANC Cross elasticity: 0.019116344
CHSUMM Own elasticity: -3.452927426
CHSUMM CHTC Cross elasticity: 0.000417852
